In [1]:
%load_ext autoreload
%autoreload 2
import os
import sys
sys.path.append("code")
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook
from sklearn.neighbors import KDTree
from plyfile import PlyData, PlyElement
from ply import write_ply
import collections

In [2]:
def compute_voxel_dico(points, voxel_size):
    
    voxel_indices = (points / voxel_size).astype(int)
    dico = dict()
    for vox in np.unique(voxel_indices, axis = 0):
        dico[tuple(vox)] = []
    for i, vox in tqdm_notebook(enumerate(voxel_indices)):
        dico[tuple(vox)] += [i]
        
    return dico

def get_cov(points):
    mean = np.mean(points, axis = 0)
    return (points - mean).transpose().dot((points - mean)) / points.shape[0]

input_dir = "../Benchmark/test_10_classes"
output_dir = input_dir + "_subsampled_2"
columns = ["x", "y", "z", "reflectance"]
voxel_size = .05

for path in os.listdir(input_dir):
    if(path[-4:] == ".ply"):
        
        
        old_cloud_path = os.path.join(input_dir, path)
        new_cloud_path = os.path.join(output_dir, path)
        print(path)
        
        data = PlyData.read(old_cloud_path)
        data = np.array([data.elements[0].data[i] for i in columns[:len(data.elements[0].properties)]]).transpose()
        if(columns[-1] == "class"):
            cloud, labels = data[:, :4], data[:, -1]
        else:
            cloud = data[:, :4]
        del data
        
        """
        #print("COMPUTING TREE")
        #tree = KDTree(cloud[:, :3], metric = "euclidean")
        #print("DONE")
        
        for i in range(3):
            plt.hist(cloud[:, i])
            plt.title(np.max(cloud[:, i]) - np.min(cloud[:, i]))
            plt.show()
        """
        
        
        dico = compute_voxel_dico(cloud[:, :3], voxel_size)
        subsampled_cloud = np.array([np.mean(cloud[vox[1]], axis = 0) for vox in tqdm_notebook(dico.items())])
        
        if(columns[-1] == "class"):
            subsampled_labels = np.array([collections.Counter(labels[vox[1]]).most_common()[0][0] for vox in tqdm_notebook(dico.items())])
            write_ply(new_cloud_path, [subsampled_cloud, subsampled_labels], columns)
        else:
            write_ply(new_cloud_path, [subsampled_cloud], columns)
          
        
        """
        eigen_values, eigen_vectors = np.linalg.eigh(get_cov(cloud[:, :2]))
        direction = eigen_vectors[:, np.argmax(eigen_values)]
        center = np.mean(cloud[:, :2], axis = 0)
        
        print(np.max(cloud, axis = 0) - np.min(cloud, axis = 0), center, direction)
        split = np.sum((cloud[:, :2] - center) * direction, axis = -1) > 0
        
        if(columns[-1] == "class"):
            print(new_cloud_path[:-4] + "_0.ply")
            write_ply(new_cloud_path[:-4] + "_0.ply", [cloud[split], labels[split]], columns)
            write_ply(new_cloud_path[:-4] + "_1.ply", [cloud[~split], labels[~split]], columns)
        else:
            write_ply(new_cloud_path[:-4] + "_0.ply", [cloud[split]], columns)
            write_ply(new_cloud_path[:-4] + "_1.ply", [cloud[~split]], columns)
        """

ajaccio_2.ply



ajaccio_57.ply



dijon_9.ply
